In [2]:
using Geodesy
include("mathies.jl")

0.3141592653589793

In [3]:
myspot = LLA( 30.228711, 92.027100 )
ced404 = LLA( 30.228678, 92.027272 )
ber266 = LLA( 30.204461, 92.031397 )
# for 3-D transformation; put into cartesian coordinates 
myspotECEF = ECEF( myspot, wgs84 )

ECEF(-195096.58265667254, 5.512084465685038e6, 3.1923052497591996e6)

In [35]:
n = Z + Y 
o = Z - Y
p = X + Z
q = X - Z
r = Y + X
s = Y - X

nfi =     Z + Φ * Y
nfe = φ * Z + Φ * Y

ofe = Φ * Z - φ * Y 


3-element Array{Float64,1}:
  0.0               
 -0.6180339887498949
  1.618033988749895 